In [1]:
import numpy as np
import csv
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier, KNeighborsRegressor
from sklearn.preprocessing import LabelEncoder


#reads in the data
data = pd.read_csv('movie_metadata.csv')

#removes the values that aren't numeric
to_drop = ['director_name', 'num_critic_for_reviews', 'actor_2_name','actor_1_name', 'movie_title', 'num_voted_users'
           , 'actor_3_name', 'plot_keywords', 'movie_imdb_link', 'num_user_for_reviews']

#makes the new data set without the to_drop colums
features_list = data.columns.difference(to_drop)
movie_data = data[features_list]
#print(np.sum(movie_num.isnull()))
#print(movie_data.content_rating.unique())
pd.options.mode.chained_assignment = None 

        
print('Done')

Done


# Changing Genres into individual Columns

In [2]:
genres = []

for string in movie_data['genres']:
    genre = string.split('|')
    genres = genres+genre
    
genres_set = set(genres)
print(genres_set)

set(['Sci-Fi', 'Crime', 'Romance', 'Animation', 'Music', 'Comedy', 'War', 'Horror', 'Film-Noir', 'Adventure', 'News', 'Reality-TV', 'Thriller', 'Western', 'Mystery', 'Short', 'Drama', 'Action', 'Documentary', 'Musical', 'History', 'Family', 'Fantasy', 'Game-Show', 'Sport', 'Biography'])


In [3]:
genres_dict = dict()
for genre in genres_set:
    genres_dict[genre] = []
    
for string in movie_data.genres:
    genres = string.split('|')
    for genre in genres_set:
        if genre in genres:
            genres_dict[genre] = genres_dict[genre]+[1]
        else:
            genres_dict[genre] = genres_dict[genre]+[0]

del movie_data['genres']
#print(genres_dict['Short'])

for genre in genres_set:
    series = pd.Series(genres_dict[genre])
    movie_data[genre] = series

#print(movie_data['Short'].values)    
print('Done')

Done


# Deleting NaN's

In [4]:
NA_THRESH = 4

#deletes the row when there are more or equal to the threshod number of NaN's
def remove_too_many_NaN(data, threshold):
    print("Length before deletion: {}".format(len(data)))

    remove_indices = []
    for index, nNaN in data.isnull().sum(axis=1).iteritems():
        if nNaN >= threshold:
            remove_indices.append(index)
    
    # drop movies with too many NaNs
    data = data.drop(data.index[remove_indices])
    print("Length after deletion: {}".format(len(data)))
    
    return data

#returns length
movie_data = remove_too_many_NaN(movie_data, NA_THRESH)

Length before deletion: 5043
Length after deletion: 4900


# Replacing NaNs with averages

In [5]:
def replace_NaNs(col):
    # compute average
    avg = np.sum(col) / (len(col) - np.sum(col.isnull()))
    print(avg)
    
    # replace NaNs with average
    col = col.fillna(value=avg)
    return col

pd.options.mode.chained_assignment= None

to_replace_NaNs = ['actor_1_facebook_likes', 'actor_2_facebook_likes', 'actor_3_facebook_likes', 
               'aspect_ratio', 'budget', 'cast_total_facebook_likes', 'director_facebook_likes']
for column in to_replace_NaNs:
    movie_data[column] = replace_NaNs(movie_data[column])

print(movie_data)

6648.62979592
1684.1759902
653.779595175
2.1284344634
39892924.8304
9849
692.662510221
      actor_1_facebook_likes  actor_2_facebook_likes  actor_3_facebook_likes  \
0                     1000.0                   936.0              855.000000   
1                    40000.0                  5000.0             1000.000000   
2                    11000.0                   393.0              161.000000   
3                    27000.0                 23000.0            23000.000000   
5                      640.0                   632.0              530.000000   
6                    24000.0                 11000.0             4000.000000   
7                      799.0                   553.0              284.000000   
8                    26000.0                 21000.0            19000.000000   
9                    25000.0                 11000.0            10000.000000   
10                   15000.0                  4000.0             2000.000000   
11                   18000.0     

# Changing Color to Numeric

In [6]:
#replaces the color value to 1
movie_data.color = movie_data.color.replace(to_replace = 'Color', value = 1)
movie_data.color = movie_data.color.replace(to_replace = 'NaN', value = 1)

#replaces the NaN or black and white values to 0
for item in movie_data.color:
    if item != 1:
        movie_data.color = movie_data.color.replace(to_replace = item, value = 0)
        
#movie_data.color = new_color_column 
#print(movie_data['color'])

#makes sure that there is nog error where it shouldn't be
pd.options.mode.chained_assignment= None
print('Done')

Done


# Changing Country to Numeric

In [7]:
#replaces the USA values to 1
movie_data.country = movie_data.country.replace(to_replace ='USA', value = 1)
movie_data.country = movie_data.country.replace(to_replace ='NaN', value = 1)
#replaces the NaN or non USA values to 0
for item in movie_data.country:
    if item != 1:
        movie_data.country = movie_data.country.replace(to_replace = item, value = 0)
       
#print(movie_data['country'])

#makes sure that there is nog error where it shouldn't be
pd.options.mode.chained_assignment= None

print('Done')

Done


# Changing Languange to Numeric

In [8]:
#replaces the English values to 1
movie_data.language = movie_data.language.replace(to_replace = 'English', value = 1)
movie_data.language = movie_data.language.replace(to_replace = 'NaN', value = 1)

#replaces the other values to 0
for item in movie_data.language:
    if item != 1:
        movie_data.language = movie_data.language.replace(to_replace = item, value = 0)
        
#print(movie_data['language'])

#makes sure that there is nog error where it shouldn't be
pd.options.mode.chained_assignment= None
print('Done')

Done


# Chaning Content_Rating to Numeric

In [9]:
def content_to_numerical(data):
    data = data.replace(to_replace='G', value=0)
    data = data.replace(to_replace='PG', value=12)
    data = data.replace(to_replace='PG-13', value=13)
    data = data.replace(to_replace='R', value=17)
    data = data.replace(to_replace='NC-17', value=17)
    
    data = data.replace(to_replace='TV-PG', value=12)
    data = data.replace(to_replace='TV-MA', value=17)
    data = data.replace(to_replace='TV-G', value=0)
    data = data.replace(to_replace='TV-Y', value=0)
    data = data.replace(to_replace='TV-Y7', value=7)
    data = data.replace(to_replace='TV-14', value=14)
    
    data = data.replace(to_replace='Not Rated', value=0)
    data = data.replace(to_replace='Unrated', value=0)
    data = data.replace(to_replace='Approved', value=0)
    data = data.replace(to_replace='Passed', value=0)
    
    data = data.replace(to_replace='X', value=17)
    data = data.replace(to_replace='M', value=17)
    data = data.replace(to_replace='GP', value=12)
    
    return data

movie_data = content_to_numerical(movie_data)
print(movie_data)

      actor_1_facebook_likes  actor_2_facebook_likes  actor_3_facebook_likes  \
0                     1000.0                   936.0              855.000000   
1                    40000.0                  5000.0             1000.000000   
2                    11000.0                   393.0              161.000000   
3                    27000.0                 23000.0            23000.000000   
5                      640.0                   632.0              530.000000   
6                    24000.0                 11000.0             4000.000000   
7                      799.0                   553.0              284.000000   
8                    26000.0                 21000.0            19000.000000   
9                    25000.0                 11000.0            10000.000000   
10                   15000.0                  4000.0             2000.000000   
11                   18000.0                 10000.0              903.000000   
12                     451.0            

In [10]:
print(movie_data.color.describe())
print(movie_data.country.describe())
print(movie_data.language.describe())

count    4900.000000
mean        0.955918
std         0.205297
min         0.000000
25%         1.000000
50%         1.000000
75%         1.000000
max         1.000000
Name: color, dtype: float64
count    4900.000000
mean        0.757347
std         0.428731
min         0.000000
25%         1.000000
50%         1.000000
75%         1.000000
max         1.000000
Name: country, dtype: float64
count    4900.000000
mean        0.935102
std         0.246371
min         0.000000
25%         1.000000
50%         1.000000
75%         1.000000
max         1.000000
Name: language, dtype: float64
